# Managing Outputs with Output Parsers


In a production setting, outputs from language models in a predictable data structure are often desirable. Consider, for instance, developing a thesaurus application to generate a collection of alternative words relevant to the given context. Large language models (LLMs) can generate numerous suggestions for synonyms or similar terms. Below is an example of output from ChatGPT listing several words closely related to “behavior.”

    Here are some substitute words for "behavior":  
      
    Conduct  
    Manner  
    Demeanor  
    Attitude  
    Disposition  
    Deportment  
    Etiquette  
    Protocol  
    Performance  
    Actions

The challenge arises from the absence of a dynamic method to extract relevant information from the provided text. Consider splitting the response by new lines and disregarding the initial lines. However, this approach is unreliable as there’s no assurance that responses will maintain a consistent format. The list might be numbered, or it might not include an introductory line.

Output Parsers enable us to define a data structure that precisely describes what is expected from the model. In a word suggestion application, you might request a list of words or a combination of different variables, such as a word and an explanation.

Structured outputs can also be enforced through APIs, such as those provided by OpenAI models, where the model can be prompted to generate outputs following a predefined schema. For instance, you can specify a JSON schema or use a Pydantic model to ensure that the outputs conform to the expected structure, making it easier to integrate into applications that require predictable data formats. This capability will be covered in more detail in the book, where we will explore practical methods to structure and validate outputs using these techniques.

The Pydantic parser is versatile and has three unique types. However, other options are also available for less complex tasks.

**Note:**  The thesaurus application will serve as a practical example to clarify the nuances of each approach.

### PydanticOutputParser

This class instructs the model to produce its output in JSON format. The parser’s output can be treated as a list, allowing for simple indexing of the results and eliminating formatting issues.

```
💡It is important to note that not all models have the same capability to generate JSON outputs. So, it would be best to use a more powerful model (like Anthropic or OpenAI’s most recent models) to get the best result.
```
This wrapper uses the Pydantic library to define and validate data structures in Python. It allows determining the expected output structure, including its name, type, and description. For instance, a variable must hold multiple suggestions, like a list, in the thesaurus application. This is achieved by creating a class that inherits the Pydantic’s `BaseModel class`.

 Set the OPENAI_API_KEY environment variable with your API credentials.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
model_name = 'gpt-3.5-turbo'
temperature = 0.0
model = ChatOpenAI(model_name=model_name, temperature=temperature)

## Pydantic OutputParser Example

In [ ]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of recieving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
        if field[0].isnumeric():
            raise ValueError("The word can not start with numbers!")
        return field

In [ ]:
parser = PydanticOutputParser(pydantic_object=Suggestions)

Import the necessary libraries and create the `Suggestions` schema class, which consists of two components:

1.  **Expected Outputs:** Each output is defined by declaring a variable with the desired type, such as a list of strings (: List[str]) in the example code. Alternatively, it could be a single string (: str) for cases expecting a singular word or sentence as the response. It’s mandatory to provide a brief description using the Field function’s description attribute, aiding the model during inference. (An illustration of handling multiple outputs will be presented later in the book.)
2.  **Validators:** We can declare functions to validate the formatting. For instance, the provided code has a validation to ensure the first character is not a number. The function’s name is not critical, but the @validator decorator must be applied to the variable requiring validation (e.g., @validator('words')). Note that if the variable is specified as a list, the field argument within the validator function will also be a list.

We will pass the created class to the `PydanticOutputParser` wrapper to make it a `LangChain` parser object. The next step is to prepare the prompt.

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

In [ ]:
target_word="behaviour"
context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."

In [ ]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

The template variable is a string incorporating named index placeholders in the following `{variable_name}` format. The template variable defines our prompts for the model, with the anticipated formatting from the output parser and the inputs (the `{format_instructions}` placeholder will be replaced by instructions from the output parser). The `PromptTemplate` takes in the template string, specifying the type of each placeholder. These placeholders can be categorized as `input_variables`, whose values are assigned later through the `.format_prompt()` method or `partial_variables`, defined immediately.

For querying models like GPT, the prompt will be passed on LangChain’s OpenAI wrapper. (It’s important to set the `OPENAI_API_KEY` environment variables with your API key from OpenAI.) Setting the temperature value to 0 also ensures that the outcomes are consistent and reproducible.

> 💡The temperature value could be between 0 and 1, where a higher number means the model is more creative. Using larger value in production is a good practice for tasks requiring creative output.

In [ ]:
chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})

In [ ]:
parser.parse(output)

The parser object’s `parse()` function will convert the model’s string response to the format we specified. You can index through the list of words and use them in your applications. Notice the simplicity of accessing the third suggestion by calling the third index instead of dealing with a lengthy string that requires extensive preprocessing, as demonstrated in the initial example.

## Multiple Outputs

The following example demonstrates a Pydantic class designed to handle multiple outputs. It instructs the model to generate a list of words and explain the reasoning behind each suggestion.

To implement this example, replace the template variable and Suggestion class with the new code (provided below). The modifications in the prompt template force the model to elaborate on its reasoning. The updated Suggestion class introduces a new output named reasons. The validator function is also applied to modify the output, ensuring each explanation ends with a period. This example also illustrates how the validator function can be used for output manipulation.


In [ ]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

    # Throw error in case of recieving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
      for item in field:
        if item[0].isnumeric():
          raise ValueError("The word can not start with numbers!")
      return field

    @validator('reasons')
    def end_with_dot(cls, field):
      for idx, item in enumerate( field ):
        if item[-1] != ".":
          field[idx] += "."
      return field

In [ ]:
parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

In [ ]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})

In [ ]:
parser.parse(output)

## CommaSeparatedListOutputParser

This class specializes in managing comma-separated outputs, focusing on instances where the model is expected to produce a list of outputs. To use this class efficiently, start by importing the necessary module.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser()

The parser does not require any configuration. As a result, it’s less adaptable and can only be used to process comma-separated strings. We can define the object by initializing the class. The steps for writing the prompt, initializing the model, and parsing the output are as follows:

In [ ]:
template = """
Offer a list of suggestions to substitue the word '{target_word}' based the presented the following text: {context}.
{format_instructions}
"""

In [ ]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})

In [ ]:
parser.parse(output)

Although most of the sample code has been explained in the previous subsection, two areas are new. First, we explored a new style for the prompt template. Second, the model’s input is generated using `.format()` rather than `.format_prompt()`. The key difference between this code and the one in the previous section is that we no longer need to call the `.to_string()` object because the prompt is already of string type.

The final result is a list of words with some overlap with the `PydanticOutputParser` technique but with more variety. However, it is not possible to rely on the `CommaSeparatedOutputParser` **class** to elucidate the reasoning behind its output.

## StructuredOutputParser

The `StructuredOutputParser` was one of the first parsers added to the LangChain library, and it remains widely used for handling structured outputs. It can manage more complex data types, including lists and **JSON-like** structures, rather than being limited to plain text. It is useful when you want a response from the model that adheres to a predefined schema, and it can support more than one response or multiple fields.

For example, in a thesaurus application, you could define a schema where multiple substitute words are returned alongside their reasoning:

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="words", description="A substitute word based on context"),
    ResponseSchema(name="reasons", description="""the reasoning of why this word fits the context.""")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)

The `StructuredOutputParser` is commonly used in combination with libraries like Pydantic to validate and enforce the structure of data returned by the model. It offers significant flexibility for managing structured output, such as **JSON** objects, which makes it versatile for complex applications.

While the `PydanticOutputParser` offers powerful validation capabilities for more intricate schemas, and the `CommaSeparatedOutputParser` can be suitable for simpler tasks, the `StructuredOutputParser` remains a flexible and robust option when more control over output formatting is needed.

Parsers serve as robust tools for extracting information from prompts and providing a degree of validation. However, they cannot guarantee an accurate response for every use case. For example, in a scenario where an application is deployed, the model’s response to a user request is incomplete, leading the parser to generate an error. `OutputFixingParser` and `RetryOutputParser` function as **fail-safes**, adding a layer to the model’s response to rectify the mistakes.

> 💡The following approaches work with the PydanticOutputParser class since it is the only one with a validation method.

## OutputFixingParser

This method aims to fix parsing errors by examining the model’s response against the defined parser description using a large language model (LLM) to address the issue. For consistency with the rest of the book, GPT-3.5 will be used, but any compatible model will work. The first step defines the Pydantic data schema.

Here’s a typical error that might arise:

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="""list of substitue words based on context""")
    reasons: List[str] = Field(description="""the reasoning of why this word fits the context""")

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"], "reasoning": ["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}'

In [ ]:
parser.parse(missformatted_output)

The error message indicates that the parser successfully detected an error in our sample response (`missformatted_output`) due to the use of the word *reasoning* instead of the expected *reasons* key. The `OutputFixingParser` class is designed to correct such errors efficiently.


In [ ]:
from langchain.output_parsers import OutputFixingParser

outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser.parse(missformatted_output)

The `from_llm()` function requires the previous parser and a language model as input parameters. It initializes a new parser equipped with the capability to rectify output errors. In this case, it identifies and modifies the incorrectly named key to match the defined requirement.

However, it’s important to note that resolving issues with the `OutputFixingParser` class may not always be feasible. The following example demonstrates using the `OutputFixingParser` class to address an error involving a missing key.

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"]}'

outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

outputfixing_parser.parse(missformatted_output)

Observing the output, it’s clear that the model recognized the absence of the reasons key in the response but lacked the context for fixing the response. Consequently, it generated a list with a single entry, whereas the expected output was one reason per word. This limitation underscores the occasional need for a more flexible approach like the `RetryOutputParser` class.

## RetryOutputParser

There are situations where the parser requires access to both the output and the prompt to fully understand the context, as highlighted in the previous example. The first step is to define the required variables.

The subsequent codes initiate the LLM, parser, and prompt described in earlier sections.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import RetryWithErrorOutputParser

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt_template.format_prompt(target_word="behaviour", context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.")

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"]}'

In [ ]:
parser.parse(missformatted_output)

Now, the same `missformatted_output` can be addressed using the `RetryWithErrorOutputParser` class. This class takes the defined parser and a model to create a new parser object. However, the `parse_with_prompt` function responsible for fixing the parsing issue requires both the generated output and the prompt.

In [ ]:
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)

In [ ]:
retry_parser.parse_with_prompt(missformatted_output, model_input)

The results demonstrate that the `RetryOutputParser` successfully resolves the issue that the `OutputFixingParser` could not. The parser effectively guides the model to generate one reason for each word, as required.

In a production environment, the recommended approach to integrating these techniques is to employ a **try...except...** method for error handling. This strategy captures parsing errors in the except block and attempts to fix them using the mentioned classes. This approach streamlines the process and limits the number of API calls, thereby reducing associated costs.